In [102]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [103]:
raw_data = pd.read_csv('../output.csv')
# raw_data = raw_data[raw_data["Group"] != "None"]
# for column in raw_data.columns[32:]:
#     raw_data = raw_data[raw_data[column] != "None"]

In [105]:
# level 0
level_0 = ['Group','BrainSeg', 'BrainSegNotVent', 'BrainSegNotVentSurf', 
				'VentricleChoroidVol',
				'lhCortex', 'rhCortex', 'Cortex', 
				'lhCerebralWhiteMatter','rhCerebralWhiteMatter', 'CerebralWhiteMatter', 
				'SubCortGray', 'TotalGray', 
				'SupraTentorial', 'SupraTentorialNotVent', 'SupraTentorialNotVentVox',
				'Mask',	'BrainSegVol-to-eTIV', 'MaskVol-to-eTIV',
				'lhSurfaceHoles', 'rhSurfaceHoles', 'SurfaceHoles',
				'EstimatedTotalIntraCranialVol', 
				'Left-Lateral-Ventricle', 'Right-Lateral-Ventricle',
		        'Left-Inf-Lat-Vent', 'Right-Inf-Lat-Vent',
				'Left-Cerebellum-White-Matter', 'Right-Cerebellum-White-Matter',
		        'Left-Cerebellum-Cortex', 'Right-Cerebellum-Cortex',
				'Left-Thalamus-Proper', 'Right-Thalamus-Proper',
				'Left-Caudate', 'Right-Caudate',
		        'Left-Putamen', 'Right-Putamen',
				'Left-Pallidum', 'Right-Pallidum',
				'3rd-Ventricle', '4th-Ventricle', '5th-Ventricle',
				'Brain-Stem',
				'Left-Hippocampus', 'Right-Hippocampus',
				'Left-Amygdala', 'Right-Amygdala',
				'CSF',
				'Left-Accumbens-area', 'Right-Accumbens-area',
				'Left-VentralDC', 'Right-VentralDC',
				'Left-vessel', 'Right-vessel',
				'Left-choroid-plexus', 'Right-choroid-plexus',
				'WM-hypointensities', 'non-WM-hypointensities',
				'Optic-Chiasm',
				'CC_Posterior', 'CC_Mid_Posterior', 
				'CC_Central',
				'CC_Mid_Anterior', 'CC_Anterior']
# level 1
level_1 = ['Group', 'BrainSegNotVent', 'BrainSegNotVentSurf',
				'VentricleChoroidVol',
		        'lhCortex', 'rhCortex', 'Cortex',
				'lhCerebralWhiteMatter', 'rhCerebralWhiteMatter', 'CerebralWhiteMatter',
				'SubCortGray', 'TotalGray', 
				'SupraTentorialNotVent', 'SupraTentorialNotVentVox', 
				'Mask', 'BrainSegVol-to-eTIV',
		        'lhSurfaceHoles', 'SurfaceHoles',
		        'EstimatedTotalIntraCranialVol', 
				'Left-Lateral-Ventricle', 'Right-Lateral-Ventricle',
		        'Left-Inf-Lat-Vent', 'Right-Inf-Lat-Vent',
		        'Left-Cerebellum-Cortex', 'Right-Cerebellum-Cortex',
				'Left-Caudate', 'Right-Caudate',
		        'Left-Putamen', 'Right-Putamen',
				'3rd-Ventricle',
		        'Brain-Stem',
				'Left-Hippocampus', 'Right-Hippocampus',
				'Left-Amygdala', 'Right-Amygdala',
				'CSF',
		        'Left-Accumbens-area', 'Right-Accumbens-area',
				'Right-vessel',
		        'Left-choroid-plexus', 
		        'WM-hypointensities', 'non-WM-hypointensities',
		        'Optic-Chiasm', 
				'CC_Posterior',
		        'CC_Central', 
				'CC_Mid_Anterior']
# level 2
level_2 = ['Group', 'VentricleChoroidVol',
				'lhCortex', 'rhCortex', 'Cortex',
				'SubCortGray', 'TotalGray',
				'BrainSegVol-to-eTIV',
				'lhSurfaceHoles', 'SurfaceHoles',
				'Left-Lateral-Ventricle', 'Right-Lateral-Ventricle',
				'Left-Inf-Lat-Vent', 'Right-Inf-Lat-Vent',
				'Left-Putamen', 'Right-Putamen',
				'3rd-Ventricle',
				'Left-Hippocampus', 'Right-Hippocampus',
				'Left-Amygdala', 'Right-Amygdala',
				'CSF',
				'Left-Accumbens-area', 'Right-Accumbens-area',
				'Left-choroid-plexus',
				'WM-hypointensities', 'non-WM-hypointensities',
				'CC_Mid_Anterior']

roi_data = raw_data[level_0]
# roi_data.info()

In [110]:
roi_data['Group'] = roi_data['Group'].replace('CN', 0)
roi_data['Group'] = roi_data['Group'].replace('MCI', 1)
roi_data['Group'] = roi_data['Group'].replace('AD', 2)

roi_data = roi_data.assign(Group=lambda s: s['Group'].astype('int'))
roi_data = roi_data.mask(roi_data['Group'] < 0).dropna()
# roi_data.info()

In [112]:
features = roi_data.mask(roi_data.eq("None")).dropna()
features_x = features[list(filter(lambda x: x != "Group", list(features.columns)))].values
features_y = features['Group'].values

# features.info()

# SVM with 10*10 Fold Validation
## Non-linear SVM

In [113]:
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

n_fold = 10
rkf = RepeatedKFold(n_splits=n_fold, n_repeats=n_fold)
total_acc = 0.

for idx, (train_idx, test_idx) in enumerate(rkf.split(features_x)):
    train_x, test_x = features_x[train_idx], features_x[test_idx]
    train_y, test_y = features_y[train_idx], features_y[test_idx]
    
    lab_enc = preprocessing.LabelEncoder()
    train_y_c = lab_enc.fit_transform(train_y)
    test_y_c = lab_enc.fit_transform(test_y)
    
    svc = base_estimator = SVC(kernel='rbf').fit(train_x, train_y_c)
    result_svc = [find_nearest([0, 1, 2], value) for value in svc.predict(test_x)]
    acc_svc = accuracy_score(test_y_c, result_svc)
    
    total_acc += acc_svc
    print("Acc %d: %f" % (idx, acc_svc))
    
total_acc /= (n_fold*n_fold)
print("Total Acc: %f" % total_acc)

/Users/hyeyoom/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Acc 0: 0.462222


/Users/hyeyoom/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Acc 1: 0.582222


/Users/hyeyoom/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


...

Acc 97: 0.462222


/Users/hyeyoom/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Acc 98: 0.506667


/Users/hyeyoom/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Acc 99: 0.484444
Total Acc: 0.486222


## Linear SVM

In [7]:
n_fold = 10
rkf = RepeatedKFold(n_splits=n_fold, n_repeats=n_fold)
total_acc = 0.

for idx, (train_idx, test_idx) in enumerate(rkf.split(features_x)):
    train_x, test_x = features_x[train_idx], features_x[test_idx]
    train_y, test_y = features_y[train_idx], features_y[test_idx]
    
    lab_enc = preprocessing.LabelEncoder()
    train_y_c = lab_enc.fit_transform(train_y)
    test_y_c = lab_enc.fit_transform(test_y)
    
    svc = base_estimator = SVC(kernel='linear').fit(train_x, train_y_c)
    result_svc = [find_nearest([0, 1, 2], value) for value in svc.predict(test_x)]
    acc_svc = accuracy_score(test_y_c, result_svc)
    
    total_acc += acc_svc
    print("Acc %d: %f" % (idx, acc_svc))
    
total_acc /= (n_fold*n_fold)
print("Total Acc: %f" % total_acc)

KeyboardInterrupt: 

# RF with 10*10 Fold Validation

In [114]:
from sklearn.ensemble import RandomForestRegressor

n_fold = 10
rkf = RepeatedKFold(n_splits=n_fold, n_repeats=n_fold)
total_acc = 0.

for idx, (train_idx, test_idx) in enumerate(rkf.split(features_x)):
    train_x, test_x = features_x[train_idx], features_x[test_idx]
    train_y, test_y = features_y[train_idx], features_y[test_idx]
    
    rfr = RandomForestRegressor(n_estimators=100).fit(train_x, train_y)
    result_rf = [find_nearest([0, 1, 2], value) for value in rfr.predict(test_x)]
    acc_rf = accuracy_score(test_y, result_rf)
    
    total_acc += acc_rf
    print("Acc %d: %f" % (idx, acc_rf))
    
total_acc /= (n_fold*n_fold)
print("Total Acc: %f" % total_acc)

Acc 0: 0.844444
Acc 1: 0.777778
Acc 2: 0.840000
Acc 3: 0.813333
Acc 4: 0.804444
Acc 5: 0.822222
Acc 6: 0.782222
Acc 7: 0.804444
Acc 8: 0.773333
Acc 9: 0.804444
Acc 10: 0.826667
Acc 11: 0.813333
Acc 12: 0.826667
Acc 13: 0.831111
Acc 14: 0.800000
Acc 15: 0.831111
Acc 16: 0.764444
Acc 17: 0.768889
Acc 18: 0.822222
Acc 19: 0.822222
Acc 20: 0.853333
Acc 21: 0.795556
Acc 22: 0.826667
Acc 23: 0.808889
Acc 24: 0.813333
Acc 25: 0.728889
Acc 26: 0.813333
Acc 27: 0.822222
Acc 28: 0.840000
Acc 29: 0.848889
Acc 30: 0.760000
Acc 31: 0.817778
Acc 32: 0.808889
Acc 33: 0.897778
Acc 34: 0.791111
Acc 35: 0.844444
Acc 36: 0.773333
Acc 37: 0.791111
Acc 38: 0.800000
Acc 39: 0.808889
Acc 40: 0.791111
Acc 41: 0.782222
Acc 42: 0.826667
Acc 43: 0.777778
Acc 44: 0.786667
Acc 45: 0.777778
Acc 46: 0.813333
Acc 47: 0.817778
Acc 48: 0.813333
Acc 49: 0.862222
Acc 50: 0.848889
Acc 51: 0.822222
Acc 52: 0.782222
Acc 53: 0.733333
Acc 54: 0.791111
Acc 55: 0.773333
Acc 56: 0.817778
Acc 57: 0.804444
Acc 58: 0.848889
Acc 59:

## Logistic Regression

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

all_train_x, all_test_x, all_train_y, all_test_y = train_test_split(features_all_x, features_all_y, test_size=0.3)

clf = LogisticRegression(solver='lbfgs', multi_class='multinomial').fit(all_train_x, all_train_y)
result = clf.predict(all_test_x)
print(accuracy_score(all_test_y, result))

clf.coef_